In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt 
import seaborn as sns
from datetime import timedelta

In [ ]:
# Carrega o arquivo CSV em um DataFrame
dados = pd.read_csv("/content/cardio_train.csv", sep=";")
# Renomeia as colunas do DataFrame
dados.columns = ["Id", "Idade", "Genero", "Altura", "Peso", "Pressao_arterial_sistolica","Pressao_arterial_diastolica",
                 "Colesterol", "Glicose", "Fumante", "Consumo_de_alcool", "Atividade_fisica", "Doenca_cardiovascular"]
# Exibe informações sobre o DataFrame
print(dados.info())

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Função para calculo dos parametros
def calculo_estatistico(confusion_matrix):
    true_negative = confusion_matrix[0, 0]
    false_positive = confusion_matrix[0, 1]
    false_negative = confusion_matrix[1, 0]
    true_positive = confusion_matrix[1, 1]

    # Cálculo da Acurácia
    accuracy = (true_negative + true_positive) / confusion_matrix.sum()

    # Cálculo da Precisão
    precision = true_positive / (true_positive + false_positive)

    # Cálculo do Recall
    recall = true_positive / (true_positive + false_negative)

    # Cálculo do F1-score
    f1 = 2 * (precision * recall) / (precision + recall)

    print("Acurácia:", accuracy)
    print("Precisão:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)

## Tratamento de Dados

In [ ]:
# Remover a coluna "id" do DataFrame
dados.drop("Id", axis=1, inplace=True)

# Remover linhas duplicadas do DataFrame
dados.drop_duplicates(inplace=True)

# Converter a coluna "Idade" de dias para anos
dados['Idade'] = dados['Idade'].apply(lambda x: timedelta(days=x) / timedelta(days=365.25))

# Calcular o Índice de Massa Corporal (IMC) e adicionar uma nova coluna "IMC" ao DataFrame
dados["IMC"] = dados["Peso"] / (dados["Altura"]/100)**2

# Mudar ordem das colunas
dados = dados[["Idade", "Genero", "Altura", "Peso","IMC", "Pressao_arterial_sistolica",
                 "Pressao_arterial_diastolica", "Colesterol", "Glicose", "Fumante", "Consumo_de_alcool", "Atividade_fisica", "Doenca_cardiovascular"]]

dados.head()

In [ ]:
dados.describe()

In [ ]:
# Criação do filtro para selecionar linhas com base em condições específicas
filtro = ((dados["Pressao_arterial_sistolica"] > 250) | (dados["Pressao_arterial_sistolica"] < 50) | 
          (dados["Pressao_arterial_diastolica"] > 170) | (dados["Pressao_arterial_diastolica"] < 30) | 
          (dados["IMC"] < 10) | (dados["IMC"] > 50))

# Aplicação do filtro ao DataFrame para remover as linhas que não atendem às condições
dados = dados[~filtro]

# Exibição das primeiras linhas do DataFrame após a aplicação do filtro
dados.head()

In [ ]:
dados.describe()

In [ ]:
sns.set(font_scale = 1.3, rc = {'figure.figsize':(20,20)})
eixo = dados.hist(bins = 70)


Modelo

In [ ]:
from sklearn.model_selection import train_test_split

# A variável alvo (Doenca_cardiovascular) é separada das demais características
previsor = np.array(dados["Doenca_cardiovascular"])
caracteristicas = np.array(dados.drop("Doenca_cardiovascular", axis=1))

# A proporção para a divisão dos dados de teste é definida
test_size = 0.2  # 20% dos dados serão usados para teste

# A semente (random_state) é definida para garantir reprodutibilidade
random_state = 42  # Valor fixo para obter a mesma divisão dos dados toda vez que o código for executado

# Os dados são divididos em conjuntos de treinamento e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(caracteristicas, previsor, test_size=test_size, random_state=random_state)

print(f'{len(caracteristicas)}\n{len(previsor)}')
print(f'x_treino {len(x_treino)}, x_teste {len(x_teste)}')
print(f'x_treino {len(x_treino)/len(caracteristicas)}, x_teste {len(x_teste)/len(caracteristicas)}')

## Regressão Logistica

In [ ]:
from sklearn.linear_model import LogisticRegression

# Cria uma instância da classe LogisticRegression com os parâmetros especificados
funcao_reg_logis = LogisticRegression(penalty='l2', C=1.0, solver='sag', max_iter=1000)

# Treina o modelo utilizando os dados de treino
funcao_reg_logis.fit(x_treino, y_treino)

LogisticRegression(max_iter=1000, solver='sag')

In [ ]:
# Fazendo previsões com o modelo final
previsoes_log = funcao_reg_logis.predict(x_teste)

In [ ]:
from sklearn.metrics import confusion_matrix

# Calculando a matriz de confusão comparando os rótulos verdadeiros (y_teste) com as previsões (previsoes_log)
matriz_confusao_log = confusion_matrix(y_teste, previsoes_log)

print(matriz_confusao_log)

[[1978  496]
 [ 789 1646]]


In [ ]:
calculo_estatistico(matriz_confusao_log)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_teste, previsoes_log))

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Definindo os valores  para os hiperparâmetros
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],  # Valores para o número de vizinhos
    'weights': ['uniform', 'distance'],  # Opções para o peso dos vizinhos
    'metric': ['euclidean', 'manhattan', 'minkowski'],  # Métricas de distância
    'p': [1, 2]  # Valores para o parâmetro p da métrica de Minkowski
}

# Criando o objeto do modelo de KNN
modelo_knn = KNeighborsClassifier()

# Realizando a busca pelos melhores hiperparâmetros usando validação cruzada
grid_search = GridSearchCV(modelo_knn, param_grid, cv=5)

# Treinando o modelo com os dados de treinamento
grid_search.fit(x_treino, y_treino)

# Obtendo os melhores hiperparâmetros encontrados
melhores_hiperparametros = grid_search.best_params_

# Criando o modelo final com os melhores hiperparâmetros
melhor_modelo_knn = KNeighborsClassifier(**melhores_hiperparametros)

# Treinando o modelo final com todos os dados de treinamento
melhor_modelo_knn.fit(x_treino, y_treino)

In [ ]:
# Fazendo previsões com o modelo final
previsoes_knn = melhor_modelo_knn.predict(x_teste)

In [ ]:
# Calculando a matriz de confusão comparando os rótulos verdadeiros (y_teste) com as previsões (previsoes_knn)
matriz_confusao_knn = confusion_matrix(y_teste, previsoes_knn)

print(matriz_confusao_knn)

In [ ]:
calculo_estatistico(matriz_confusao_knn)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_teste, previsoes_knn))

## RANDON FLOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Definindo os valores possíveis para os hiperparâmetros
param_grid = {
    'n_estimators': [100, 200],  # Número de árvores na floresta
    'max_depth': [None, 5],  # Profundidade máxima das árvores
    'min_samples_split': [2, 5],  # Número mínimo de amostras necessárias para dividir um nó
    'min_samples_leaf': [1, 2],  # Número mínimo de amostras necessárias em um nó folha
    'max_features': ['sqrt', None]  # Número de features a serem consideradas ao procurar a melhor divisão
}
# Criando o modelo de Random Forest
modelo_rf = RandomForestClassifier()
# Realizando a busca pelos melhores hiperparâmetros usando validação cruzada
grid_search = GridSearchCV(modelo_rf, param_grid, cv=5)
# Treinando o modelo com os dados de treinamento
grid_search.fit(x_treino, y_treino)
# Obtendo os melhores hiperparâmetros encontrados
melhores_hiperparametros = grid_search.best_params_
# Criando o modelo final com os melhores hiperparâmetros
melhor_modelo_rf = RandomForestClassifier(**melhores_hiperparametros)
# Treinando o modelo final com todos os dados de treinamento
melhor_modelo_rf.fit(x_treino, y_treino)

In [ ]:
# Realizando as previsões com os dados de teste
previsoes_rf = melhor_modelo_rf.predict(x_teste)

In [ ]:
# Calculando a matriz de confusão comparando os rótulos verdadeiros (y_teste) com as previsões (previsoes_rf)
matriz_confusao_rf = confusion_matrix(y_teste, previsoes_rf)

print(matriz_confusao_rf)

In [ ]:
calculo_estatistico(matriz_confusao_rf)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_teste, previsoes_rf))